<a href="https://colab.research.google.com/github/yunyoungwoo/2024S-Ajou-ML-FP/blob/main/temp.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [1]:
import random
import torch

seed = 1
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
if torch.cuda.is_available():
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
torch.backends.cudnn.benchmark = False
torch.backends.cudnn.deterministic = True

In [ ]:
import os
from PIL import Image
from tqdm import tqdm
from torch.utils.data import DataLoader
import torchvision.transforms as transforms

In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [ ]:
train_path = "/kaggle/input/2022-ai-tp-18011886/train"
test_path = "/kaggle/input/2022-ai-tp-18011886/test"
all_label = os.listdir(train_path)

In [ ]:
class train_dataset():
    def __init__(self):
        self.data_path = []
        self.label_idx = []

        for idx, label in enumerate(all_label):
            for data_path in tqdm(os.listdir(os.path.join(train_path,label))):
                path = os.path.join(label,data_path)
                self.data_path.append(path)
                self.label_idx.append(idx)

    def __getitem__(self,idx):
        transform = transforms.Compose([
        transforms.ToTensor(),
        transforms.Resize((320,320)),
        transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))])

        path = self.data_path[idx]
        img = Image.open(os.path.join(train_path,path))
        img = img.convert('RGB')
        img = transform(img)
        return img, self.label_idx[idx]

    def __len__(self):
        return len(self.data_path)

class test_dataset():
    def __init__(self):
        self.data_path = []

        for data_path in tqdm(os.listdir(os.path.join(test_path))):
            path = data_path
            self.data_path.append(path)

        self.data_path.sort()

    def __getitem__(self,idx):
        transform = transforms.Compose([
        transforms.ToTensor(),
        transforms.Resize((320,320)),
        transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))])

        path = self.data_path[idx]
        img = Image.open(os.path.join(test_path,path))
        img = img.convert('RGB')
        img = transform(img)
        return img, 0

    def __len__(self):
        return len(self.data_path)

In [ ]:
batch_size = 32
train_dataset = train_dataset()
test_dataset = test_dataset()
train_loader = DataLoader(train_dataset, batch_size=batch_size,
                         shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size,
                         shuffle=False)

In [ ]:
import torchvision.models as models

model = models.resnet50(pretrained=True)
model.fc = torch.nn.Linear(2048, 4, bias=True)
torch.nn.init.xavier_normal_(model.fc.weight)
model.to(device)

In [ ]:
loss = torch.nn.CrossEntropyLoss().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr = 0.0005)
scheduler = torch.optim.lr_scheduler.ExponentialLR(optimizer, gamma=0.9)

In [ ]:
model.train()

epochs = 5

for epoch in range(epochs):
    avg_cost = 0

    for x,y in tqdm(train_loader):
        #import pdb;pdb.set_trace()
        x = x.to(device)
        y = y.to(device)

        optimizer.zero_grad()
        out = model(x).to(device)
        cost = loss(out, y).to(device)
        cost.backward()
        optimizer.step()
        scheduler.step()

        avg_cost +=cost.item()/batch_size

    print(epoch+1,avg_cost)

In [ ]:
with torch.no_grad():
    model.eval()
    corr_pred = list()
    test = list()
    for x,_ in test_loader:
        x = x.to(device)
        out = model(x).to(device)
        pred = torch.argmax(out,dim=1).cpu().detach().numpy()
        corr_pred.extend(pred)

    for i in range(len(corr_pred)):
        test.append(all_label[corr_pred[i]])

In [ ]:
submit = pd.read_csv('/kaggle/input/2022-ai-tp-18011886/sample_submit.csv',encoding='cp949')
submit['label'] = test
submit.to_csv('submit.csv',index=False,encoding='cp949')